## CREAZIONE DELL'AMBIENTE DI LAVORO
#### ACQUISIZIONE DEL DATASET

In [100]:
import os

# Cambiare le credenziali kaggle per poter scaricare i dati
# Si uncommenti e si cambino le stringhe con le proprie credenziali
# os.environ['KAGGLE_USERNAME'] = "your_username"
# os.environ['KAGGLE_KEY'] = "your_password"

# Download di tutti i file .csv presenti nel dataset di instgram
!kaggle datasets download -p ./data -d bhanupratapbiswas/instgram --unzip 

Dataset URL: https://www.kaggle.com/datasets/bhanupratapbiswas/instgram
License(s): ODC Public Domain Dedication and Licence (PDDL)
100%|█████████████████████████████████████████| 151k/151k [00:00<00:00, 459kB/s]
100%|█████████████████████████████████████████| 151k/151k [00:00<00:00, 458kB/s]


#### CONTAINER NEO4J

Per fare il build e lo start del container di neo4j (si parte dalla empty sandbox) si deve far girare nella cartella del progetto contente `docker-compose.yml` il comando riportato di seguito.

Si e' voluto fare un docker-compose cosi da aggiungere in maniera piu' elegante le opzioni sulla creazione del container.

```bash
sudo docker-compose up
```

**NB**: Nell'ambiente di visual studio non si riesce, purtroppo, a far girare comandi di questo tipo in modo interattivo (nessuna possibilità di immettere la password).
Si e' anche valutato se mettere la password in un file .env e richiamarla da riga di comando ma non sarebbe stata sicura.

#### CONNESSIONE AL DATABASE

In [85]:
from neo4j import GraphDatabase as gd

# Prima di arrivare a questo punto si controlli di aver correttamente attivato il container
# Nel caso in cui ci siano dubbi su ip/porta da usare si controlli nel log docker la voce "bolt"
URI = "bolt://localhost:7688" 

d   = gd.driver(URI, auth=None)
d.verify_connectivity()

#### CREAZIONE DATABASE

In [86]:
s   = d.session()

# Prova a creare un nuovo database vuoto col nome di instagram
# Che ci fosse gia' o meno alla fine setta la sessione sul database instagram
try:
    s.run("create database instagram").data() 
except: 
    s = d.session(database="instagram")
finally:
    s = d.session(database="instagram")

In [87]:
# La creazione dei constraint viene fatta per ogni nodo che sara' presente
s.run("create constraint UniqueUserID for (u:User) require u.id is unique;")
s.run("create constraint UniqueCommentID for (c:Comment) require c.id is unique;")
s.run("create constraint UniquePhotoID for (p:Photo) require p.id is unique;")
s.run("create constraint UniqueTagID for (t:Tag) require t.id is unique;")


Accedendo a http://localhost:7474 e poi con nessuna autenticazione a bolt://localhost:7688 si può:
* controllare con `:dbs` che esiste il database instagram
* con `:use instagram` entrare nel database e con `:schema` controllare che siano stati aggiunti i quattro constraint

In [43]:
# La creazione degli index viene fatta per ogni nodo in modo da avere gia' buone performance 
# su alcune delle operazioni che possono essere fatte su di essi 
# TODO: METTI GLI INDEX

In [88]:
# Si è fatto un test con un mini file csv per essere sicuri di come e dove trovare tutti i csv
# La stampa ritorna il numero di righe del file (3)
res = s.run('''LOAD CSV WITH HEADERS
                FROM 'file:///test.csv' AS line
                RETURN count(line)''').data()
print(res)

[{'map': {'beverage': 'Soda', 'name': 'Selma', 'age': '9'}}, {'map': {'beverage': 'Tea', 'name': 'Rana', 'age': '12'}}, {'map': {'beverage': 'Cola', 'name': 'Selina', 'age': '19'}}]


## RESEARCH QUESTIONS

In [ ]:
s.close()
d.close()
d.close()